In [291]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [292]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [(("-"if _i == -1 else "" if _i == 1 else (str(_i) + "*")) + n) for _i, n in zip(c[0], names) if _i != 0]
        print(c[3] + ": ([" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2]) + ")") 

In [293]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = ["1","2","3","4","5"]                        # V
customers = ["1","2","3","4","5","6","7","8","9"]       # C

num_vehicles = 5
num_customers = 6
locations = ["O"] + customers[:num_customers] + ["X"]   # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route


In [294]:
my_names = ["x"+str(i)+str(j) + str(k) for k in vehicles[:num_vehicles] for i in (["O"] + customers[:num_customers]) for j in (customers[:num_customers] + ["X"])]
num_names = len(my_names)
print(num_names)
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

245
Names  ['xO11', 'xO21', 'xO31', 'xO41', 'xO51', 'xO61', 'xOX1', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x1X1', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x2X1', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x3X1', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x4X1', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x5X1', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x6X1', 'xO12', 'xO22', 'xO32', 'xO42', 'xO52', 'xO62', 'xOX2', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x1X2', 'x212', 'x222', 'x232', 'x242', 'x252', 'x262', 'x2X2', 'x312', 'x322', 'x332', 'x342', 'x352', 'x362', 'x3X2', 'x412', 'x422', 'x432', 'x442', 'x452', 'x462', 'x4X2', 'x512', 'x522', 'x532', 'x542', 'x552', 'x562', 'x5X2', 'x612', 'x622', 'x632', 'x642', 'x652', 'x662', 'x6X2', 'xO13', 'xO23', 'xO33', 'xO43', 'xO53', 'xO63', 'xOX3', 'x113', 'x123', 'x133', 'x143', 'x153', 'x163', 'x1X3', 'x213', 'x223', 'x233', 'x243', 'x253', 'x263', 'x2X3', 'x313', 'x323', 'x333', 'x343', 'x35

In [295]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
__v = lambda n: 0 if n == "O" or n == "X" else int(n)
my_obj = [int(cost_matrix.iat[__v(n[1]),__v(n[2]) + 1]) for n in my_names]

print("Object ", my_obj)

Object  [18, 30, 28, 27, 8, 18, 0, 0, 23, 10, 4, 18, 22, 18, 23, 0, 32, 29, 18, 10, 30, 10, 32, 0, 2, 20, 31, 28, 4, 29, 2, 0, 19, 28, 27, 18, 18, 20, 19, 0, 5, 8, 22, 10, 31, 28, 5, 0, 18, 18, 30, 28, 27, 8, 18, 0, 0, 23, 10, 4, 18, 22, 18, 23, 0, 32, 29, 18, 10, 30, 10, 32, 0, 2, 20, 31, 28, 4, 29, 2, 0, 19, 28, 27, 18, 18, 20, 19, 0, 5, 8, 22, 10, 31, 28, 5, 0, 18, 18, 30, 28, 27, 8, 18, 0, 0, 23, 10, 4, 18, 22, 18, 23, 0, 32, 29, 18, 10, 30, 10, 32, 0, 2, 20, 31, 28, 4, 29, 2, 0, 19, 28, 27, 18, 18, 20, 19, 0, 5, 8, 22, 10, 31, 28, 5, 0, 18, 18, 30, 28, 27, 8, 18, 0, 0, 23, 10, 4, 18, 22, 18, 23, 0, 32, 29, 18, 10, 30, 10, 32, 0, 2, 20, 31, 28, 4, 29, 2, 0, 19, 28, 27, 18, 18, 20, 19, 0, 5, 8, 22, 10, 31, 28, 5, 0, 18, 18, 30, 28, 27, 8, 18, 0, 0, 23, 10, 4, 18, 22, 18, 23, 0, 32, 29, 18, 10, 30, 10, 32, 0, 2, 20, 31, 28, 4, 29, 2, 0, 19, 28, 27, 18, 18, 20, 19, 0, 5, 8, 22, 10, 31, 28, 5, 0, 18]


In [296]:
capacity_matrix   = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[1000, 70, 145, 90, 50]

In [297]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{'1': 32, '2': 45, '3': 16, '4': 15, '5': 22, '6': 50}

In [298]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [299]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
constraint_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if name[1] != name[2]  and name[1] == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        constraint_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(constraint_2)


C21: ([x121 x131 x141 x151 x161 x1X1 x122 x132 x142 x152 x162 x1X2 x123 x133 x143 x153 x163 x1X3 x124 x134 x144 x154 x164 x1X4 x125 x135 x145 x155 x165 x1X5]  E  1)
C22: ([x211 x231 x241 x251 x261 x2X1 x212 x232 x242 x252 x262 x2X2 x213 x233 x243 x253 x263 x2X3 x214 x234 x244 x254 x264 x2X4 x215 x235 x245 x255 x265 x2X5]  E  1)
C23: ([x311 x321 x341 x351 x361 x3X1 x312 x322 x342 x352 x362 x3X2 x313 x323 x343 x353 x363 x3X3 x314 x324 x344 x354 x364 x3X4 x315 x325 x345 x355 x365 x3X5]  E  1)
C24: ([x411 x421 x431 x451 x461 x4X1 x412 x422 x432 x452 x462 x4X2 x413 x423 x433 x453 x463 x4X3 x414 x424 x434 x454 x464 x4X4 x415 x425 x435 x455 x465 x4X5]  E  1)
C25: ([x511 x521 x531 x541 x561 x5X1 x512 x522 x532 x542 x562 x5X2 x513 x523 x533 x543 x563 x5X3 x514 x524 x534 x544 x564 x5X4 x515 x525 x535 x545 x565 x5X5]  E  1)
C26: ([x611 x621 x631 x641 x651 x6X1 x612 x622 x632 x642 x652 x6X2 x613 x623 x633 x643 x653 x6X3 x614 x624 x634 x644 x654 x6X4 x615 x625 x635 x645 x655 x6X5]  E  1)


In [300]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
constraint_3 = []
val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if n1 != "O" and n1 != n2 else c if n1 == "O" and n2 != "O" else 0
# val = lambda v, c, n1, n2, n3: 0 if n3 != v else int(demand[n1]) if n1 != "O" and n1 != n2 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    constraint_3.append(([val(veh, float(cap), name[1], name[2], name[3]) for name in my_names],"G", 0, "C3" + str(_i)))


display(constraint_3)

C31: ([1000.0*xO11 1000.0*xO21 1000.0*xO31 1000.0*xO41 1000.0*xO51 1000.0*xO61 1000.0*xOX1 -32*x121 -32*x131 -32*x141 -32*x151 -32*x161 -32*x1X1 -45*x211 -45*x231 -45*x241 -45*x251 -45*x261 -45*x2X1 -16*x311 -16*x321 -16*x341 -16*x351 -16*x361 -16*x3X1 -15*x411 -15*x421 -15*x431 -15*x451 -15*x461 -15*x4X1 -22*x511 -22*x521 -22*x531 -22*x541 -22*x561 -22*x5X1 -50*x611 -50*x621 -50*x631 -50*x641 -50*x651 -50*x6X1]  G  0)
C32: ([70.0*xO12 70.0*xO22 70.0*xO32 70.0*xO42 70.0*xO52 70.0*xO62 70.0*xOX2 -32*x122 -32*x132 -32*x142 -32*x152 -32*x162 -32*x1X2 -45*x212 -45*x232 -45*x242 -45*x252 -45*x262 -45*x2X2 -16*x312 -16*x322 -16*x342 -16*x352 -16*x362 -16*x3X2 -15*x412 -15*x422 -15*x432 -15*x452 -15*x462 -15*x4X2 -22*x512 -22*x522 -22*x532 -22*x542 -22*x562 -22*x5X2 -50*x612 -50*x622 -50*x632 -50*x642 -50*x652 -50*x6X2]  G  0)
C33: ([145.0*xO13 145.0*xO23 145.0*xO33 145.0*xO43 145.0*xO53 145.0*xO63 145.0*xOX3 -32*x123 -32*x133 -32*x143 -32*x153 -32*x163 -32*x1X3 -45*x213 -45*x233 -45*x243 -45

In [301]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
constraint_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    constraint_4.append(([(1 if (name[1] == "O" and name[3] == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
display(constraint_4)

C41: ([xO11 xO21 xO31 xO41 xO51 xO61 xOX1]  E  1)
C42: ([xO12 xO22 xO32 xO42 xO52 xO62 xOX2]  E  1)
C43: ([xO13 xO23 xO33 xO43 xO53 xO63 xOX3]  E  1)
C44: ([xO14 xO24 xO34 xO44 xO54 xO64 xOX4]  E  1)
C45: ([xO15 xO25 xO35 xO45 xO55 xO65 xOX5]  E  1)


In [302]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
constraint_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if n[3] != k or n[2] == n[1] else 1 if n[1] == p else -1 if n[2] == p else 0                
        constraint_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(constraint_5) 

C51: ([-xO11 x121 x131 x141 x151 x161 x1X1 -x211 -x311 -x411 -x511 -x611]  E  0)
C52: ([-xO21 -x121 x211 x231 x241 x251 x261 x2X1 -x321 -x421 -x521 -x621]  E  0)
C53: ([-xO31 -x131 -x231 x311 x321 x341 x351 x361 x3X1 -x431 -x531 -x631]  E  0)
C54: ([-xO41 -x141 -x241 -x341 x411 x421 x431 x451 x461 x4X1 -x541 -x641]  E  0)
C55: ([-xO51 -x151 -x251 -x351 -x451 x511 x521 x531 x541 x561 x5X1 -x651]  E  0)
C56: ([-xO61 -x161 -x261 -x361 -x461 -x561 x611 x621 x631 x641 x651 x6X1]  E  0)
C57: ([-xO12 x122 x132 x142 x152 x162 x1X2 -x212 -x312 -x412 -x512 -x612]  E  0)
C58: ([-xO22 -x122 x212 x232 x242 x252 x262 x2X2 -x322 -x422 -x522 -x622]  E  0)
C59: ([-xO32 -x132 -x232 x312 x322 x342 x352 x362 x3X2 -x432 -x532 -x632]  E  0)
C510: ([-xO42 -x142 -x242 -x342 x412 x422 x432 x452 x462 x4X2 -x542 -x642]  E  0)
C511: ([-xO52 -x152 -x252 -x352 -x452 x512 x522 x532 x542 x562 x5X2 -x652]  E  0)
C512: ([-xO62 -x162 -x262 -x362 -x462 -x562 x612 x622 x632 x642 x652 x6X2]  E  0)
C513: ([-xO13 x123 x133 x

In [303]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
constraint_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    constraint_6.append(([(1 if (name[3] == veh and name[2] == "X") else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
display(constraint_6)


C61: ([xOX1 x1X1 x2X1 x3X1 x4X1 x5X1 x6X1]  E  1)
C62: ([xOX2 x1X2 x2X2 x3X2 x4X2 x5X2 x6X2]  E  1)
C63: ([xOX3 x1X3 x2X3 x3X3 x4X3 x5X3 x6X3]  E  1)
C64: ([xOX4 x1X4 x2X4 x3X4 x4X4 x5X4 x6X4]  E  1)
C65: ([xOX5 x1X5 x2X5 x3X5 x4X5 x5X5 x6X5]  E  1)


In [304]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
constraint_x = []
_i = 0

_i += 1
constraint_x.append(([(1 if name[1] == name[2] else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(constraint_x)


CX1: ([x111 x221 x331 x441 x551 x661 x112 x222 x332 x442 x552 x662 x113 x223 x333 x443 x553 x663 x114 x224 x334 x444 x554 x664 x115 x225 x335 x445 x555 x665]  E  0)


In [305]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
constraint_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            constraint_y.append(([(1 if ((name[1] == i and j == name[2]) or (name[1] == j and i == name[2])) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(constraint_y)


CX1: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX2: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX3: ([x141 x411 x142 x412 x143 x413 x144 x414 x145 x415]  L  1)
CX4: ([x151 x511 x152 x512 x153 x513 x154 x514 x155 x515]  L  1)
CX5: ([x161 x611 x162 x612 x163 x613 x164 x614 x165 x615]  L  1)
CX6: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX7: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX8: ([x241 x421 x242 x422 x243 x423 x244 x424 x245 x425]  L  1)
CX9: ([x251 x521 x252 x522 x253 x523 x254 x524 x255 x525]  L  1)
CX10: ([x261 x621 x262 x622 x263 x623 x264 x624 x265 x625]  L  1)
CX11: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX12: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX13: ([x341 x431 x342 x432 x343 x433 x344 x434 x345 x435]  L  1)
CX14: ([x351 x531 x352 x532 x353 x533 x354 x534 x355 x535]  L  1)
CX15: ([x361 x631 x362 x632 x363 x633 x364 x634 x365 x635]  L  1)
CX16: ([x141 x411 x

In [306]:
def foo(n : int, a : list, o : list):
    temp = {}
    tempa = []
    for ai in a:
        _i =  np.prod([ord(c) + 99 for c in ai])
        if _i not in temp:            
            temp[_i] = ai
            tempa.append(ai)
            
    a = tempa
    arr = []
    if n > 1:
        for ai in a:
            for oi in o:
                if oi not in ai:
                    arr.append(ai + oi)
        return foo(n-1, arr, o)
    else:
        return a

def get_path(n: int, a: list) -> list:
    return foo(n,a,a)


In [307]:
# # CONSTRAINT_Z 
constraint_z = []
_i = 0

for r in range(3,num_customers + 1):
    for p in get_path(r, customers[:num_customers]):
#         p = "134"
#         r = 3
        _i += 1
        constraint_z.append(([1 if name[1] in p and name[2] in p and name[1] != name[2] else 0 for name in my_names], "L", r-1, "CZ" + str(_i)))

display(constraint_z)


CZ1: ([x121 x131 x211 x231 x311 x321 x122 x132 x212 x232 x312 x322 x123 x133 x213 x233 x313 x323 x124 x134 x214 x234 x314 x324 x125 x135 x215 x235 x315 x325]  L  2)
CZ2: ([x121 x141 x211 x241 x411 x421 x122 x142 x212 x242 x412 x422 x123 x143 x213 x243 x413 x423 x124 x144 x214 x244 x414 x424 x125 x145 x215 x245 x415 x425]  L  2)
CZ3: ([x121 x151 x211 x251 x511 x521 x122 x152 x212 x252 x512 x522 x123 x153 x213 x253 x513 x523 x124 x154 x214 x254 x514 x524 x125 x155 x215 x255 x515 x525]  L  2)
CZ4: ([x121 x161 x211 x261 x611 x621 x122 x162 x212 x262 x612 x622 x123 x163 x213 x263 x613 x623 x124 x164 x214 x264 x614 x624 x125 x165 x215 x265 x615 x625]  L  2)
CZ5: ([x131 x141 x311 x341 x411 x431 x132 x142 x312 x342 x412 x432 x133 x143 x313 x343 x413 x433 x134 x144 x314 x344 x414 x434 x135 x145 x315 x345 x415 x435]  L  2)
CZ6: ([x131 x151 x311 x351 x511 x531 x132 x152 x312 x352 x512 x532 x133 x153 x313 x353 x513 x533 x134 x154 x314 x354 x514 x534 x135 x155 x315 x355 x515 x535]  L  2)
CZ7: ([x13

In [308]:
# Merge segment constraints
constraints_group = []
constraints_group += constraint_2 
constraints_group += constraint_3 
constraints_group += constraint_4 
constraints_group += constraint_5 
constraints_group += constraint_6
constraints_group += constraint_x
constraints_group += constraint_y
constraints_group += constraint_z


# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [309]:
# Create Cplex Problems
myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")


myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)
    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 124)

In [310]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    for v, n in zip(values, names):
        if v != 0:
            print(n, " - ",v)
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")


Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 16 rows and 30 columns.
MIP Presolve modified 188 coefficients.
Reduced MIP has 108 rows, 215 columns, and 3195 nonzeros.
Reduced MIP has 215 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.30 ticks)
Found incumbent of value 180.000000 after 0.00 sec. (4.40 ticks)
Probing time = 0.02 sec. (2.64 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 108 rows, 215 columns, and 3195 nonzeros.
Reduced MIP has 215 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.66 ticks)
Probing time = 0.00 sec. (2.66 ticks)
Clique table members: 1708.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (1.46 ticks)

        Nodes                           